In [1]:
import time
import datetime
import pandas as pd
from pandas import DataFrame
import numpy as np
from numpy import array
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, make_scorer, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from math import exp
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
Xcsv = pd.read_csv('out_featers.csv', index_col='match_id')

In [3]:
n_samples = Xcsv.shape[0]#1000#
X = Xcsv[0:n_samples]
X = X.fillna(0)#, inplace=True)
Y = X['radiant_win']
X.drop(['radiant_win',
        'duration',
        'tower_status_radiant', 
        'tower_status_dire', 
        'barracks_status_dire', 
        'barracks_status_radiant', 
        'start_time'],
       axis=1, inplace=True)


In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.3,
    random_state = 241 )
scaler = MinMaxScaler()
X_t = scaler.fit_transform(X_train)

3.0
-0.3981934785376382


In [6]:
logreg = LogisticRegression(n_jobs=-1, solver='saga')
%time logreg.fit(X_t, Y_train);
X_tt = scaler.transform(X_test)
s = logreg.score(X_tt, Y_test)
print(s)

Wall time: 9.31 s
0.6563817751722719


In [8]:
cv = KFold(n_splits=5, shuffle=True, random_state=241)
scorer = make_scorer(roc_auc_score)
params = {'C' : [i/20 for i in range(10, 16)]}
%time grid = GridSearchCV(LogisticRegression(random_state=241, solver='saga', n_jobs=-1), params, cv=cv, n_jobs=-1, scoring=scorer, verbose=True)
grid.fit(X_t, Y_train);

Wall time: 0 ns
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.0min finished


In [9]:
grid.best_params_

{'C': 0.75}

{{grid.best_params_}}

# 1 
Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [10]:
col = [name for name in Xcsv.columns.values if '_hero' in name] + ['lobby_type']
#col.append('lobby_type')
Xm = X.drop(col, axis=1)


In [11]:

X_train, X_test, Y_train, Y_test = train_test_split(
    Xm, Y, test_size = 0.3,
    random_state = 241 )
scaler = StandardScaler()
X_t = scaler.fit_transform(X_train)
logreg = LogisticRegression(C=.75,
                            random_state=241,
                            solver='saga',
                            n_jobs=-1)
logreg.fit(X_t, Y_train);
X_tt = scaler.transform(X_test)
s = logreg.score(X_tt, Y_test)
print(s)

0.65624464328568


На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?


Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).


[Coursera](www.coursera.com)